- 1. 层和块

In [2]:
import torch
from torch import nn

# Relu: rectified linear unit 修正线性单元
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.randn(2, 20)
print(net(X))

tensor([[ 0.0062, -0.1392,  0.2363,  0.0832,  0.0243,  0.1784, -0.1006, -0.3280,
         -0.1614,  0.3042],
        [-0.2247, -0.1395,  0.1103,  0.1092, -0.1485,  0.0350, -0.1509, -0.1952,
         -0.0477, -0.1296]], grad_fn=<AddmmBackward0>)


- 2. 自定义块

In [5]:
from torch.nn import functional as F

class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10)

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # hidden_relu = nn.ReLU(self.hidden(X)) # TypeError: linear(): argument 'input' (position 1) must be Tensor, not ReLU
        hidden_relu = F.relu(self.hidden(X))
        return self.out(hidden_relu)

net = MLP()
X = torch.randn(2, 20)
print(net(X))

tensor([[-0.1740,  0.1725,  0.0923,  0.1195, -0.5226,  0.1948, -0.0990,  0.4433,
          0.4237, -0.2867],
        [-0.2174,  0.2314, -0.0177,  0.1429,  0.1685, -0.4289,  0.0787,  0.1734,
         -0.4437,  0.1642]], grad_fn=<AddmmBackward0>)


- 3. 顺序块

In [6]:
import torch
from torch import nn

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args): # enumerate()函数 将 索引和元素打包成元组
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module # 有序字典

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for module in self._modules.values():
            X = module(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.randn(2, 20)
print(net(X))

tensor([[-0.7123,  0.2012, -0.5300, -0.1819, -0.1099, -0.1140,  0.4888,  0.5346,
         -0.5012, -0.0522],
        [-0.1565, -0.0852, -0.3347, -0.0873, -0.2728, -0.3032,  0.2280,  0.0763,
         -0.1513,  0.0404]], grad_fn=<AddmmBackward0>)


- 4. 在前向传播函数中执行代码

In [9]:
from torch import nn
from torch.nn import functional as F

class FixHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        print('X', X)
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixHiddenMLP()
X = torch.randn(2, 20)
print(net(X))

X tensor([[ 0.3810,  0.9665, -0.3679, -1.4356,  0.1412,  0.2186, -0.1983, -0.1711,
          0.5136,  0.5563, -0.5663, -2.0419,  0.2936,  1.1343,  1.3069, -0.2156,
         -1.2543,  0.0417, -0.9581, -1.6532],
        [-0.2604, -0.3824, -0.9645,  0.8524, -1.0908,  0.4778, -1.2645, -1.6788,
         -0.3141, -0.4336, -2.6020, -0.3865, -1.4434, -0.2953, -0.0914,  0.2393,
         -0.0936,  0.6398,  1.4310, -0.1027]])
tensor(-0.1768, grad_fn=<SumBackward0>)


# 可以混合搭配各种组合块的方法

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU()
                                 )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixHiddenMLP())

X = torch.randn(2, 20)
print(chimera(X))

X tensor([[ 2.5135e-01, -1.3667e-01,  2.4794e-01,  1.4598e-02, -1.5441e-01,
         -4.6506e-02, -2.0240e-02,  1.4868e-01,  9.2175e-02,  1.7458e-01,
          1.0291e-01, -2.2695e-01,  1.8266e-01, -2.6253e-01, -2.0054e-01,
          1.2006e-01, -1.0820e-01, -1.2115e-01,  6.0168e-02, -5.5276e-05],
        [ 1.3988e-01, -1.0401e-01,  1.7652e-01,  2.0877e-03, -8.2170e-02,
         -1.5255e-02, -6.9511e-02,  1.4035e-01,  1.5811e-02,  2.4414e-01,
          6.8375e-02, -1.3626e-01,  4.8868e-02, -1.3627e-01, -1.5445e-01,
          1.9012e-01, -1.2681e-01, -1.5437e-01,  1.0323e-01,  1.1605e-02]],
       grad_fn=<AddmmBackward0>)
tensor(-0.0109, grad_fn=<SumBackward0>)
